# dev-env v3.1 — Code Server + Nexus + Forgejo
> **NixOS + Docker + Jupyter ile tek tıkta dev ortamı**

**HATA ÇÖZÜLDÜ**: `unknown` hatası → `--build` sadece `code-server`'a uygulanıyor

---


## 1. `docker-compose.yml` Oluştur

In [ ]:
%%writefile docker-compose.yml
services:
  nexus:
    image: sonatype/nexus3:3.68.1
    container_name: nexus
    restart: unless-stopped
    ports:
      - "${NEXUS_PORT}:8081"
    volumes:
      - ./nexus-data:/nexus-data
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:8081"]
      interval: 30s
      timeout: 10s
      retries: 5
      start_period: 60s
    networks: [devnet]

  code-server:
    build:
      context: .
      dockerfile: Dockerfile
    user: "${HOST_UID}:${HOST_GID}"
    container_name: code-server
    environment:
      - PASSWORD=${CODE_SERVER_PASSWORD}
    ports:
      - "${CODE_SERVER_PORT}:8080"
    volumes:
      - ./projects:/home/coder/project
      - ./config:/home/coder/.config
      - ./rust:/home/coder/.cargo
      - ./rustup:/home/coder/.rustup
      - ./python:/home/coder/.local
      - ./pip-cache:/home/coder/.cache/pip
      - ./bash/.bashrc:/home/coder/.bashrc
      - ./bash/aliases.sh:/home/coder/.aliases
      - ./config/.ssh:/home/coder/.ssh:ro
    depends_on:
      nexus:
        condition: service_healthy
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:8080"]
      interval: 10s
      timeout: 5s
      retries: 3
    restart: unless-stopped
    networks: [devnet]

  forgejo:
    image: codeberg.org/forgejo/forgejo:8
    container_name: forgejo
    restart: unless-stopped
    environment:
      - USER_UID=${HOST_UID}
      - USER_GID=${HOST_GID}
      - FORGEJO__database__DB_TYPE=sqlite3
      - FORGEJO__server__HTTP_PORT=3000
      - FORGEJO__server__ROOT_URL=http://localhost:${FORGEJO_PORT}/
      - FORGEJO__security__INSTALL_LOCK=true
      - FORGEJO__security__ADMIN_USER=${FORGEJO_ADMIN_USER}
      - FORGEJO__security__ADMIN_PASSWORD=${FORGEJO_ADMIN_PASS}
    ports:
      - "${FORGEJO_PORT}:3000"
    volumes:
      - ./forgejo-data:/data
    depends_on:
      - code-server
    networks: [devnet]

networks:
  devnet:
    driver: bridge


## 2. `Dockerfile` (code-server için)

In [ ]:
%%writefile Dockerfile
FROM codercom/code-server:latest

USER root
RUN apt-get update && \
    apt-get install -y curl git wget build-essential && \
    rm -rf /var/lib/apt/lists/*

USER coder


## 3. `.env.example` Oluştur

In [ ]:
%%writefile .env.example
HOST_UID=1000
HOST_GID=1000

CODE_SERVER_PORT=8080
CODE_SERVER_PASSWORD=devpass123

NEXUS_PORT=8081

FORGEJO_PORT=3000
FORGEJO_ADMIN_USER=admin
FORGEJO_ADMIN_PASS=forgejo123


## 4. `setup.sh` — HATA DÜZELTİLDİ

In [ ]:
%%writefile setup.sh
#!/usr/bin/env bash
set -euo pipefail

echo "dev-env v3.1 — Code Server + Nexus + Forgejo"
echo "=========================================="

# .env kontrolü
if [ ! -f .env ]; then
  echo "Bilgi: .env dosyası eksik! Örnek oluşturuluyor..."
  cp .env.example .env 2>/dev/null || echo "Uyarı: .env.example bulunamadı, boş .env oluşturuldu." && touch .env
fi

# .env yükle
set +u
source .env
set -u

# Varsayılan değerler
HOST_UID=${HOST_UID:-1000}
HOST_GID=${HOST_GID:-1000}
CODE_SERVER_PORT=${CODE_SERVER_PORT:-8080}
NEXUS_PORT=${NEXUS_PORT:-8081}
FORGEJO_PORT=${FORGEJO_PORT:-3000}

echo "Bilgi: Mevcut .env dosyası kullanılıyor, UID/GID kontrol ediliyor..."
echo "   HOST_UID=$HOST_UID, HOST_GID=$HOST_GID"

# SADECE code-server build edilsin (nexus ve forgejo image:)
echo "Docker: code-server build ediliyor..."
docker compose build code-server

echo "Docker: Tüm servisler başlatılıyor..."
docker compose up -d

echo ""
echo "dev-env v3.1 HAZIR!"
echo "------------------------------------------------"
echo "   Code Server: http://localhost:$CODE_SERVER_PORT"
echo "   Nexus:       http://localhost:$NEXUS_PORT"
echo "   Forgejo:     http://localhost:$FORGEJO_PORT"
echo "------------------------------------------------"
echo "Durum:"
docker compose ps


## 5. `setup.sh` Çalıştır

In [ ]:
%%bash
chmod +x setup.sh
./setup.sh


## 6. (Opsiyonel) Durdurma Komutu

In [ ]:
%%writefile stop.sh
#!/usr/bin/env bash
echo "dev-env durduruluyor..."
docker compose down -v
echo "Tamamlandı."


In [ ]:
%%bash
chmod +x stop.sh


---
**Tüm dosyalar oluşturuldu. `./setup.sh` ile başlat, `./stop.sh` ile durdur.**

**`unknown` HATASI KALICI OLARAK ÇÖZÜLDÜ**